<a href="https://colab.research.google.com/github/liwenjing2012/Cousera_Capstone/blob/main/Capstone_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cousera Capstone Project : Neighborhoods Battle
Where is better to live in Austin

In [3]:
#install modules needed
!pip install rtree
!pip install geopandas

     |████████████████████████████████| 1.0MB 4.3MB/s 
     |████████████████████████████████| 1.0MB 4.2MB/s 
     |████████████████████████████████| 15.4MB 245kB/s 
     |████████████████████████████████| 6.6MB 34.7MB/s 


In [4]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
from shapely.geometry import Point
import geopandas as gpd
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup
# import k-means from clustering stage
from sklearn.cluster import KMeans

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [ ]:
CLIENT_ID = "KU0CC2XGBFTS40SCQZYT5C1W4JMNIRA1D5H3SVZCITNDO3XF"
CLIENT_SECRET="HCOL1SIL4HGTJKTXPG34GVCF0YIPNY4PIOTVLDTERRFQRZYD"
VERSION = '20180604'
ACCESS_TOKEN = '0GFYVQWD4RZ2YPEWEFDYCKM0FN5ADDL1KVYPC1PWASSNHZBH'

latitude = 40.715337
longitude = -74.008848
radius = 500
LIMIT = 100
search_query = 'housing'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url
# send GET request and get trending venues
results = requests.get(url).json()
results

In [ ]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  after removing the cwd from sys.path.


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4bbfd504461576b057547932,New York City Housing Court,"[{'id': '4bf58dd8d48988d12b941735', 'name': 'C...",v-1623413012,False,111 Centre St,btwn Leonard & White,40.716811,-74.002015,"[{'label': 'display', 'lat': 40.71681123524968...",599,10013,US,New York,NY,United States,"[111 Centre St (btwn Leonard & White), New Yor..."
1,4e8c9ecc4901755a2e292434,Housing conference room,"[{'id': '4bf58dd8d48988d100941735', 'name': 'M...",v-1623413012,False,NaN,NaN,40.714114,-74.009358,"[{'label': 'display', 'lat': 40.71411438661994...",142,10007,US,Manhattan,NY,United States,"[Manhattan, NY 10007]"
2,51aa9f36498e6e87c2f3c43e,"housing works chambers st, nyc","[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",v-1623413012,False,NaN,NaN,40.714169,-74.006472,"[{'label': 'display', 'lat': 40.71416887391371...",238,NaN,US,New York,NY,United States,"[New York, NY]"
3,509819dbe4b0e52fa55ebd47,New York City Housing Authority,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1623413012,False,90 Church Street,NaN,40.711893,-74.011019,"[{'label': 'display', 'lat': 40.71189329396107...",424,NY,US,New York,NY,United States,"[90 Church Street, New York, NY NY]"
4,4e4abe4afa76a0c058d425ca,Housing Resources Administration,"[{'id': '4bf58dd8d48988d126941735', 'name': 'G...",v-1623413012,False,NaN,NaN,40.718037,-74.005707,"[{'label': 'display', 'lat': 40.71803665161133...",400,NaN,US,New York,NY,United States,"[New York, NY]"


In [ ]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Pisillo Italian Panini,Sandwich Place,97 Nassau St,at Ann St,40.710530,-74.007526,"[{'label': 'display', 'lat': 40.71053008446078...",546,10038,US,New York,NY,United States,"[97 Nassau St (at Ann St), New York, NY 10038]",528bf16711d2b7722da6b51c
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,NaN,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",124,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007]",4f3232e219836c91c7bfde94
2,Caruso's Italian Restaurant,Italian Restaurant,42 Broadway,NaN,40.710509,-74.008276,"[{'label': 'display', 'lat': 40.71050909999999...",539,10004,US,New York,NY,United States,"[42 Broadway, New York, NY 10004]",4e4c4d28bd413c4cc66857a5
3,Pisillo Italian Cafe,Café,99 Nassau St,NaN,40.710458,-74.007693,"[{'label': 'display', 'lat': 40.710458, 'lng':...",551,10038,US,New York,NY,United States,"[99 Nassau St, New York, NY 10038]",6065ecdcc85e724bffef8c08
4,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,NaN,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",497,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282]",4fa862b3e4b0ebff2f749f06
5,Gloria's Restaurant,Italian Restaurant,107 W Broadway,Reade St,40.715794,-74.008789,"[{'label': 'display', 'lat': 40.71579360961914...",51,10013,US,New York,NY,United States,"[107 W Broadway (Reade St), New York, NY 10013]",4c5de89e6ebe2d7f7955d42e


Import criminal report data from github

In [5]:
url = "https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/Crime_Reports.csv"
criminal_data = pd.read_csv(url)
criminal_data.head()


,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,Location Type,Address,Zip Code,Council District,APD Sector,APD District,PRA,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
0,20201530258,ROBBERY BY ASSAULT,303,N,06/01/2020 03:30:00 AM,06/01/2020,330,06/01/2020 03:58:00 AM,06/01/2020,358,BAR / NIGHT CLUB,9621 N IH 35 SVRD NB,78753.0,4.0,ED,7,241.0,18.33,N,06/18/2020,120,Robbery,3131655.0,3131655.0,30.361035,-97.685308,"(30.36103465, -97.68530777)"
1,20201530887,FAMILY DISTURBANCE,3400,N,06/01/2020 02:02:00 PM,06/01/2020,1402,06/01/2020 02:02:00 PM,06/01/2020,1402,RESIDENCE / HOME,2302 SWEET CLOVER DR,78745.0,5.0,FR,1,525.0,17.28,NaN,NaN,NaN,NaN,3090537.0,3090537.0,30.190364,-97.820164,"(30.19036406, -97.82016354)"
2,20201530201,BURGLARY NON RESIDENCE,502,N,06/01/2020 03:22:00 AM,06/01/2020,322,06/01/2020 03:22:00 AM,06/01/2020,322,DRUG STORE / DR. OFFICE / HOSPITAL,1909 1/2 E 38TH ST,78723.0,1.0,BA,4,900.0,3.00,N,06/03/2020,220,Burglary,3125696.0,3125696.0,30.287689,-97.706205,"(30.28768858, -97.70620522)"
3,20205021293,BURGLARY OF VEHICLE,601,N,06/01/2020 07:50:00 AM,06/01/2020,750,06/01/2020 10:11:00 AM,06/01/2020,1011,RESIDENCE / HOME,408 KREBS LN,78704.0,3.0,DA,2,474.0,NaN,N,06/02/2020,23F,Theft,3107193.0,3107193.0,30.227088,-97.766456,"(30.22708779, -97.76645599)"
4,20205021334,BURGLARY OF VEHICLE,601,N,05/30/2020 11:00:00 PM,05/30/2020,2300,06/01/2020 12:22:00 PM,06/01/2020,1222,PARKING LOTS / GARAGE,3105 N LAMAR BLVD,78705.0,9.0,BA,3,345.0,2.00,N,06/02/2020,23F,Theft,3113056.0,3113056.0,30.300634,-97.745917,"(30.30063372, -97.74591661)"


In [28]:
criminal_features = ['Incident Number','Highest Offense Description', 'Category Description','Occurred Date', 'Location Type', 'Zip Code', 'Latitude', 'Longitude']
criminal_feature_data = criminal_data[criminal_features]
criminal_feature_data = criminal_feature_data[-criminal_feature_data['Category Description'].isna()].reset_index(drop=True)
criminal_feature_data.groupby(['Category Description']).count()

,Incident Number,Highest Offense Description,Occurred Date,Location Type,Zip Code,Latitude,Longitude
Category Description,,,,,,,
Aggravated Assault,2789,2789,2789,2786,2763,2738,2738
Auto Theft,4019,4019,4019,4019,3996,3998,3998
Burglary,4824,4824,4824,4824,4810,4800,4800
Murder,57,57,57,57,57,57,57
Rape,557,557,557,555,523,0,0
Robbery,1064,1064,1064,1064,1060,1057,1057
Theft,26712,26712,26712,26709,26583,26624,26624


In [36]:
criminal_feature_data.shape

(40022, 9)

Import neighborhood data from github and merge the two datasets

In [33]:
url = 'https://raw.githubusercontent.com/liwenjing2012/Cousera_Capstone/main/Neighborhoods.geojson'
geoNeigh = gpd.read_file(url)
geoNeigh_data = geoNeigh[['neighname','geometry']]
geoNeigh_data



In [37]:
criminal_gdf = gpd.GeoDataFrame(
    criminal_feature_data, geometry=gpd.points_from_xy(criminal_feature_data.Longitude, criminal_feature_data.Latitude))

In [40]:
criminal_gdf.set_crs(epsg=4326, inplace=True)
joined_gdf = gpd.sjoin(criminal_gdf, geoNeigh_data, op='within').reset_index(drop=True).drop(['index_right'],axis=1)

In [47]:
joined_gdf.head()

,Incident Number,Highest Offense Description,Category Description,Occurred Date,Location Type,Zip Code,Latitude,Longitude,geometry,neighname
0,20201530258,ROBBERY BY ASSAULT,Robbery,06/01/2020,BAR / NIGHT CLUB,78753.0,30.361035,-97.685308,POINT (-97.68531 30.36103),WINDSOR HILLS
1,20201530395,BURGLARY OF RESIDENCE,Burglary,06/01/2020,RESIDENCE / HOME,78753.0,30.357407,-97.673489,POINT (-97.67349 30.35741),WINDSOR HILLS
2,20201550308,BURGLARY OF VEHICLE,Theft,06/03/2020,HOTEL / MOTEL / ETC.,78753.0,30.374727,-97.676837,POINT (-97.67684 30.37473),WINDSOR HILLS
3,20201550210,BURGLARY NON RESIDENCE,Burglary,06/03/2020,GAS / SERVICE STATIONS,78753.0,30.375772,-97.675650,POINT (-97.67565 30.37577),WINDSOR HILLS
4,20201570291,MAIL THEFT,Theft,06/05/2020,OTHER / UNKNOWN,78753.0,30.352493,-97.675258,POINT (-97.67526 30.35249),WINDSOR HILLS


In [58]:
df1 = joined_gdf.groupby(['neighname'])['Incident Number'].count().reset_index(name="count")

In [59]:
kclusters = 3

df_clustering = df1.drop(['neighname'],1)
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(df_clustering)
kmeans.labels_
df1.insert(0,'clusters',kmeans.labels_)
df1

,clusters,neighname,count
0,0,ALLANDALE,357
1,1,ANDERSON MILL,507
2,1,AVERY RANCH--LAKELINE,675
3,0,BARTON CREEK MALL,257
4,0,BARTON HILLS,352
...,...,...,...
97,0,WINDSOR HILLS,369
98,2,WINDSOR PARK,1062
99,0,WINDSOR ROAD,93
100,0,WOOTEN,299


In [61]:
!pip install googlemaps
import googlemaps


  Created wheel for googlemaps: filename=googlemaps-4.4.5-cp37-none-any.whl size=37995 sha256=4bcf84648fc56321f8cf05511d22ea00ef92100775eb31eecbdb7438388b7734
  Stored in directory: /root/.cache/pip/wheels/fb/24/33/51b0faba1170f6519eae3c51b6adbd9939296c73f54230b069
Successfully built googlemaps
